In [ ]:
# Clone GitHub repository
!git clone https://github.com/zlisto/social_media_marketing

import os
os.chdir("social_media_marketing")


In [ ]:
# Install requirements
!pip install openai


In [46]:
import pandas as pd
import datetime
import openai
import textwrap as tr
import os 
import time
import ast

pd.set_option("display.max_colwidth", None)

In [1]:
#set your OpenAI API key as a string
OPENAI_API_KEY= ''


In [26]:
client = openai.Client(api_key=OPENAI_API_KEY)


# Create YaleSOM Tweets

In [27]:
instruction_file_path = "data/instructions_YaleSOM.txt"
with open(instruction_file_path, 'r', encoding='utf-8') as file:
    instructions = file.read()


In [29]:
print(instructions)

You are going to act like the YaleSOM Twitter account for the Yale School of Management.  Some top tweets from the account are provided at the end.  Learn from theset tweets what types of tweets are good and the style of YaleSOM tweeting.  You will be asked by the user to write engaging tweets in the voice of this account. You may also be asked to write the tweet on a user specificed topic.  Provide the tweet in your response and make the tweet be engaging, based on your analysis of past YaleSOM tweets provided here.  Your response should be the tweet, along with a short explanation why it will be engaging.  If the topic is not represented in these tweets, use your best knowledge to craft the tweet.



"Prof. @JeffSonnenfeld on whether Elon Musk's eccentricities are undermining his leadership of Twitter, Tesla, and SpaceX.

Read more in #Yaleinsights: https://t.co/LYGPdPJJaG"
"Want to learn about antitrust, but don't know where to start? In a series of engaging animated videos, Prof. @

In [83]:
#create assistant
assistant_name = 'YaleSOM GPT'
model = 'gpt-3.5-turbo-1106'
#model="gpt-4-1106-preview"
print(f"{assistant_name} Model {model}")

assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=instructions,
    model=model
)



YaleSOM GPT Model gpt-3.5-turbo-1106


In [84]:
#function to chat with assistant

def get_chatbot_response(user_input, thread):
    # This is where you'd integrate your chatbot logic
    print("Getting GPT response...")
    message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input,
            file_ids=file_ids
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

        if run_status.completed_at is not None:
            break

        time.sleep(1)  # Sleep for a second

    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )

    # Assistant's response
    msg = messages.data[0]
    
    if msg.role == 'assistant':
        for content in msg.content:
            if content.type == 'text':
                response  = content.text.value
            else:
                response = "No text in response"

    return f"{response}" 

In [88]:
thread = client.beta.threads.create()  #initialize the conversation thread


In [89]:
#chat with assistant
while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        break
    response = get_chatbot_response(user_input, thread)
    print(tr.fill(f"\n{assistant_name}: {response}", width = 75))


You: write a tweet about Fanta orange soda
Getting GPT response...

YaleSOM GPT: 🎉 Can't resist a tangy Fanta? You're not alone! Join the Fanta lovers club as we celebrate the zing and zest of Fanta orange soda. Not a part of the crew? It's never too late to embrace the bubbly, citrusy goodness. #FantaLove 🍊💥

You: write a tweet as YaleSOM about fanta orange soda
Getting GPT response...

YaleSOM GPT: 🍊 Embrace the zing and zest! Fanta orange soda brings a burst of citrusy refreshment. From its effervescent fizz to its vibrant orange flavor, Fanta is more than just a drink; it's a celebration in a bottle. Join us in savoring the delightful, bubbly essence of Fanta! #FantaLove

You: quit


# Image Generation

In [78]:
#Give instructions to the AI
instruction = "you will help create cool looking sneakers for a high end fashion company catering to hip young people."
instruction+="You return the answers as prompts for Dalle-3 as JSON output with keys prompt1, prompt2, etc."

#Give your task for the AI
user_input =  "Give me 5 ideas for images of sneaers that we can post on the companys instagram."



In [79]:
#get the response from the AI
responses = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": instruction},
    {"role": "user", "content": user_input}
  ]
)

response = responses.choices[0].message.content
print(response)


{
  "prompt1": "Create a sleek and modern sneaker design with bold contrasting colors and clean lines for a chic urban look.",
  "prompt2": "Design a vibrant and eye-catching sneaker featuring intricate patterns and textures inspired by street art and graffiti culture.",
  "prompt3": "Develop a minimalist yet luxurious sneaker with opulent metallic accents and embossed logo detailing for a touch of sophistication.",
  "prompt4": "Craft a retro-inspired sneaker silhouette with a modern twist, incorporating nostalgic 90s color schemes and playful geometric shapes.",
  "prompt5": "Fashion an edgy and daring high-top sneaker design with unconventional materials and avant-garde embellishments to appeal to the fashion-forward crowd."
}


In [82]:
#convert the text response into a json object
prompt_json = ast.literal_eval(response.strip())


In [81]:
#generate the image for each prompt
for cnt,prompt in enumerate(prompt_json.values()):
    print(tr.fill(f"{cnt+1}. {prompt}", width = 75))
    response_img = client.images.generate(
      model="dall-e-3",
      prompt=prompt,
      size="1024x1024",
      quality="standard",
      n=1,
    )
    time.sleep(1)

    image_url = response_img.data[0].url
    print(image_url)

1. Create a sleek and modern sneaker design with bold contrasting colors and clean lines for a chic urban look.
https://oaidalleapiprodscus.blob.core.windows.net/private/org-zrRGnR9FWbvUcS3XEz3mQZCr/user-l84oQyDEGesRPUmssaSpPuwX/img-pn0eX7kgKzsybFA1QYzB3whu.png?st=2023-11-27T08%3A03%3A36Z&se=2023-11-27T10%3A03%3A36Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-11-27T05%3A59%3A02Z&ske=2023-11-28T05%3A59%3A02Z&sks=b&skv=2021-08-06&sig=q5SfrH/Cq/PYX65i9OjGGAlDk1MmgeGFwhAECfBfGgM%3D
2. Design a vibrant and eye-catching sneaker featuring intricate patterns and textures inspired by street art and graffiti culture.
https://oaidalleapiprodscus.blob.core.windows.net/private/org-zrRGnR9FWbvUcS3XEz3mQZCr/user-l84oQyDEGesRPUmssaSpPuwX/img-fHeVsYWZHTX0JU99GZrIE6nd.png?st=2023-11-27T08%3A03%3A50Z&se=2023-11-27T10%3A03%3A50Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a